In [41]:
import pyspark

In [42]:
myConf=pyspark.SparkConf()
spark=pyspark.sql.SparkSession.builder.master("local").appName("myApp").config(conf=myConf).getOrCreate()

## 1) DataFrame을 생성

In [43]:
df = spark\
    .read\
    .options(header='false', inferschema='true', delimiter='\t')\
    .csv('축사.txt')\
    .withColumnRenamed("_c0", "문장")

In [44]:
df.show()

+----------------------------------+
|                              문장|
+----------------------------------+
|    제74주년 경찰의 날 기념식 축사|
|    연설자 : 국무총리 연설일 : ...|
|       글자작게 글자크게 인쇄 목록|
|  존경하는 국민 여러분, 경찰관 ...|
| 국민의 안전을 위해 밤낮없이 애...|
|오늘 홍조근정훈장을 받으신 중앙...|
|           사랑하는 경찰관 여러분,|
|여러분의 헌신적 노력으로 우리의...|
| 치안의 개선은 국민의 체감으로 ...|
| 한국을 찾는 외국 관광객들도 우...|
|   올해는 ‘경찰의 날’에 맞춰 국...|
|         자랑스러운 경찰관 여러분,|
| 경찰헌장은 “나라와 겨레를 위하...|
|    대한민국 경찰은 1945년 광복...|
| 임시정부 초대 경무국장 백범 김...|
|    광복 이후 6·25전쟁에서도 경...|
|그러나 잘못도 없지는 않았습니다...|
| 지금 경찰은 과거를 돌아보며 국...|
| 검경 수사권 조정과 자치경찰제 ...|
|정부는 경찰의 근무여건을 개선하...|
+----------------------------------+
only showing top 20 rows



## 2) 단어로 분리해서, 출력

In [45]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="문장", outputCol="단어")

In [46]:
tokdf = tokenizer.transform(df)

In [47]:
tokdf.show()

+----------------------------------+--------------------------------+
|                              문장|                            단어|
+----------------------------------+--------------------------------+
|    제74주년 경찰의 날 기념식 축사|    [제74주년, 경찰의, 날, 기...|
|    연설자 : 국무총리 연설일 : ...|   [연설자, :, 국무총리, 연설...|
|       글자작게 글자크게 인쇄 목록|[글자작게, 글자크게, 인쇄, 목록]|
|  존경하는 국민 여러분, 경찰관 ...|   [존경하는, 국민, 여러분,, ...|
| 국민의 안전을 위해 밤낮없이 애...|  [국민의, 안전을, 위해, 밤낮...|
|오늘 홍조근정훈장을 받으신 중앙...|[오늘, 홍조근정훈장을, 받으신...|
|           사랑하는 경찰관 여러분,|     [사랑하는, 경찰관, 여러분,]|
|여러분의 헌신적 노력으로 우리의...| [여러분의, 헌신적, 노력으로,...|
| 치안의 개선은 국민의 체감으로 ...|  [치안의, 개선은, 국민의, 체...|
| 한국을 찾는 외국 관광객들도 우...|  [한국을, 찾는, 외국, 관광객...|
|   올해는 ‘경찰의 날’에 맞춰 국...|    [올해는, ‘경찰의, 날’에, ...|
|         자랑스러운 경찰관 여러분,|   [자랑스러운, 경찰관, 여러분,]|
| 경찰헌장은 “나라와 겨레를 위하...| [경찰헌장은, “나라와, 겨레를...|
|    대한민국 경찰은 1945년 광복...|    [대한민국, 경찰은, 1945년...|
| 임시정부 초대 경무국장 백범 김...|  [임시정부, 초대, 경무국장, ...|
|    광복 이후 6·25전쟁에서도 경...|    [광복, 이후, 6·25전쟁에서...|
|그러나 잘못도 없지는 않았습니다...|

##  3) 불용어 구성, 출력 - 축사 전문에서 한글자로 된 단어를 찾아내 스스로 구성

In [48]:
from pyspark.ml.feature import StopWordsRemover
stop = StopWordsRemover(inputCol="단어", outputCol="nostops")

stopwords=[u"참",u"그", u"돌",u"큰",u"법",u"더",u"등"]
stop.setStopWords(stopwords)

StopWordsRemover_22119d93ceec

In [49]:
for e in stop.getStopWords():
    print (e, end="/")

참/그/돌/큰/법/더/등/

## 4) 불용어 제거하고, 출력

In [50]:
stopdf=stop.transform(tokdf) 
stopdf.show()

+----------------------------------+--------------------------------+--------------------------------+
|                              문장|                            단어|                         nostops|
+----------------------------------+--------------------------------+--------------------------------+
|    제74주년 경찰의 날 기념식 축사|    [제74주년, 경찰의, 날, 기...|    [제74주년, 경찰의, 날, 기...|
|    연설자 : 국무총리 연설일 : ...|   [연설자, :, 국무총리, 연설...|   [연설자, :, 국무총리, 연설...|
|       글자작게 글자크게 인쇄 목록|[글자작게, 글자크게, 인쇄, 목록]|[글자작게, 글자크게, 인쇄, 목록]|
|  존경하는 국민 여러분, 경찰관 ...|   [존경하는, 국민, 여러분,, ...|   [존경하는, 국민, 여러분,, ...|
| 국민의 안전을 위해 밤낮없이 애...|  [국민의, 안전을, 위해, 밤낮...|  [국민의, 안전을, 위해, 밤낮...|
|오늘 홍조근정훈장을 받으신 중앙...|[오늘, 홍조근정훈장을, 받으신...|[오늘, 홍조근정훈장을, 받으신...|
|           사랑하는 경찰관 여러분,|     [사랑하는, 경찰관, 여러분,]|     [사랑하는, 경찰관, 여러분,]|
|여러분의 헌신적 노력으로 우리의...| [여러분의, 헌신적, 노력으로,...| [여러분의, 헌신적, 노력으로,...|
| 치안의 개선은 국민의 체감으로 ...|  [치안의, 개선은, 국민의, 체...|  [치안의, 개선은, 국민의, 체...|
| 한국을 찾는 외국 관광객들도 우...|  [한국을, 찾는, 외국, 관광객...|  [한국을, 찾는, 외국

## 5) TF-IDF를 계산하고, 출력

In [51]:
from pyspark.ml.feature import HashingTF, IDF

In [52]:
hashTF = HashingTF(inputCol="nostops", outputCol="hash")
hashdf = hashTF.transform(stopdf)
idf = IDF(inputCol="hash", outputCol="idf")
idfModel = idf.fit(hashdf)
idfdf = idfModel.transform(hashdf)

In [53]:
for e in idfdf.select("nostops","idf").take(5):
    print(e)

Row(nostops=['제74주년', '경찰의', '날', '기념식', '축사'], idf=SparseVector(262144, {66920: 2.4849, 80697: 1.5686, 91663: 2.4849, 144010: 2.4849, 165348: 2.4849}))
Row(nostops=['연설자', ':', '국무총리', '연설일', ':', '2019.10.21'], idf=SparseVector(262144, {50106: 2.4849, 91275: 2.4849, 188482: 2.4849, 188646: 2.4849, 231152: 4.9698}))
Row(nostops=['글자작게', '글자크게', '인쇄', '목록'], idf=SparseVector(262144, {41140: 2.4849, 58897: 2.4849, 128576: 2.4849, 132302: 2.4849}))
Row(nostops=['존경하는', '국민', '여러분,', '경찰관', '여러분,', '일흔네', '‘경찰의', '날’입니다.'], idf=SparseVector(262144, {162: 2.4849, 80732: 2.4849, 118171: 3.1372, 127225: 2.4849, 142775: 2.0794, 160086: 1.3863, 172380: 2.4849}))
Row(nostops=['국민의', '안전을', '위해', '밤낮없이', '애쓰시는', '전국의', '15만', '경찰관', '여러분께', '먼저', '감사를', '드립니다.', '전몰·순직', '경찰관들의', '고귀한', '희생에', '경의를', '표합니다.', '유가족', '여러분께', '위로의', '마음을', '전합니다.'], idf=SparseVector(262144, {5341: 2.4849, 6304: 2.4849, 30732: 2.4849, 39431: 1.7918, 43098: 2.4849, 49855: 2.0794, 51468: 2.4849, 63600: 2.4849, 75300:


## 6) TF-IDF 컬럼을 features로 구성, 출력

In [55]:
idfdf=idfdf.withColumnRenamed('idf','features')
idfdf.select('features').show(5,truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                                                                                                                                                                                                                                                                   